<a href="https://colab.research.google.com/github/VincentChen2/BasketBet-AI-NAISS-2024-Spring-/blob/main/Test_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nba_api

In [8]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, commonplayerinfo
import pandas as pd

#Imports for Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [9]:
def get_player_id(player_name: str) -> str:
    for p in players.get_players():
        if p['full_name'].lower() == player_name.lower():
            return p['id']

    raise NameError('Invalid Player Name; Player Name Does Not Exist')
    return None

def get_player_career_start_year(player_id: str) -> int:
    player_info = commonplayerinfo.CommonPlayerInfo(player_id = player_id)
    career_start_year = player_info.get_data_frames()[0]['FROM_YEAR'][0]

    return int(career_start_year)

player_id = get_player_id('stephen curry')
get_player_career_start_year(player_id)

2009

# Interactive Player Selection


In [11]:
player_name = input('Enter Name: ')
player_id = get_player_id(player_name)
start_year = get_player_career_start_year(player_id)
all_seasons_data = pd.DataFrame()

for year in range(start_year, 2024): #Change last parameter if you're not in 2024
    season_str = str(year) + '-' + str(year + 1)[-2:]
    gamelog = playergamelog.PlayerGameLog(player_id = player_id, season = season_str)
    season_data = gamelog.get_data_frames()[0]
    all_seasons_data = pd.concat([all_seasons_data, season_data], ignore_index = True)

all_seasons_data['OPPONENT_TEAM'] = all_seasons_data['MATCHUP'].apply(lambda x: x.split(' ')[-1])
all_seasons_data['OPPONENT_TEAM']

Enter Name: stephen curry


0      POR
1      UTA
2      OKC
3      LAC
4      MIN
      ... 
927    SAC
928    NOP
929    HOU
930    SAC
931    PHX
Name: OPPONENT_TEAM, Length: 932, dtype: object

# Encoding

In [15]:
encoder = OneHotEncoder(sparse_output = False)
opponent_team_encoded = encoder.fit_transform(all_seasons_data[['OPPONENT_TEAM']])
opponent_team_df = pd.DataFrame(opponent_team_encoded, columns = encoder.get_feature_names_out(['OPPONENT_TEAM']))
opponent_team_df

,OPPONENT_TEAM_ATL,OPPONENT_TEAM_BKN,OPPONENT_TEAM_BOS,OPPONENT_TEAM_CHA,OPPONENT_TEAM_CHI,OPPONENT_TEAM_CLE,OPPONENT_TEAM_DAL,OPPONENT_TEAM_DEN,OPPONENT_TEAM_DET,OPPONENT_TEAM_HOU,...,OPPONENT_TEAM_OKC,OPPONENT_TEAM_ORL,OPPONENT_TEAM_PHI,OPPONENT_TEAM_PHX,OPPONENT_TEAM_POR,OPPONENT_TEAM_SAC,OPPONENT_TEAM_SAS,OPPONENT_TEAM_TOR,OPPONENT_TEAM_UTA,OPPONENT_TEAM_WAS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
df_encoded = all_seasons_data.join(opponent_team_df)
feature_columns = [col for col in df_encoded.columns if 'OPPONENT_TEAM_' in col]
feature_columns

['OPPONENT_TEAM_ATL',
 'OPPONENT_TEAM_BKN',
 'OPPONENT_TEAM_BOS',
 'OPPONENT_TEAM_CHA',
 'OPPONENT_TEAM_CHI',
 'OPPONENT_TEAM_CLE',
 'OPPONENT_TEAM_DAL',
 'OPPONENT_TEAM_DEN',
 'OPPONENT_TEAM_DET',
 'OPPONENT_TEAM_HOU',
 'OPPONENT_TEAM_IND',
 'OPPONENT_TEAM_LAC',
 'OPPONENT_TEAM_LAL',
 'OPPONENT_TEAM_MEM',
 'OPPONENT_TEAM_MIA',
 'OPPONENT_TEAM_MIL',
 'OPPONENT_TEAM_MIN',
 'OPPONENT_TEAM_NJN',
 'OPPONENT_TEAM_NOH',
 'OPPONENT_TEAM_NOP',
 'OPPONENT_TEAM_NYK',
 'OPPONENT_TEAM_OKC',
 'OPPONENT_TEAM_ORL',
 'OPPONENT_TEAM_PHI',
 'OPPONENT_TEAM_PHX',
 'OPPONENT_TEAM_POR',
 'OPPONENT_TEAM_SAC',
 'OPPONENT_TEAM_SAS',
 'OPPONENT_TEAM_TOR',
 'OPPONENT_TEAM_UTA',
 'OPPONENT_TEAM_WAS']

# Model

In [36]:
score_threshold = float(input('Input number of points to score: '))
X = df_encoded[feature_columns]
Y = (df_encoded['PTS'] > score_threshold).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, Y_train)

Input number of points to score: 20


RandomForestClassifier()

# Prediction

In [37]:
opponent_team = input('Team to play against: ')

input_data = pd.DataFrame(columns = feature_columns)
input_data.loc[0, :] = 0
input_data.loc[0, f'OPPONENT_TEAM_{opponent_team.upper()}'] = 1

Team to play against: BKN


In [43]:
prediction = random_forest_model.predict(input_data)
prediction_probability = random_forest_model.predict_proba(input_data)
prediction_result = 'Yes' if prediction[0] == 1 else 'No'
probability = prediction_probability[0][prediction[0]]
print(f'Prediction: {prediction_result}\nProbability: {round(probability * 100, 3)}%')

Prediction: Yes
Probability: 79.677%
